In [ ]:
# OS : macOS-10.16-x86_64-i386-64bit
# Python : 3.8.18
# Numpy : 1.24.3
# Pandas : 2.0.3
# Matplotlib : 3.7.2
# Seaborn : 0.12.2
# Scikit-learn : 1.2.2
# xgboost : 1.7.3
# lightgbm : 3.3.5
# catboost : 1.1.1
# xlrd : 2.0.1
# openpyxl : 3.0.10
# ipykernel : 6.25.0
# notebook : 6.5.4
# graphviz : 0.20.1
# Created: OCT. 30. 2023
# Author: D.W. SHIN


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

from xgboost import XGBClassifier

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score


In [ ]:
dataset = load_breast_cancer()
x_data = dataset.data
y_data = dataset.target
columns = dataset.feature_names

cancer_df = pd.DataFrame(data= x_data, columns=columns)
cancer_df.head()

In [ ]:
# target 값이 악성과 양성의 갯수가 어떻게 될까?
cancer_df['target'] = y_data
cancer_df['target'].value_counts()

### EDA

In [ ]:
# 정상적인 데이터의 갯수 확인하기
msno.bar(cancer_df, figsize=(16,5), color='lightblue')
plt.show()

#### 결측치를 만들어 봅시다.

In [ ]:
temp_df = cancer_df.copy()
temp_df.head()

In [ ]:
# 결측치 만들기
# temp_df.iloc[24:27, [1,9]] = np.nan
# temp_df.iloc[13, 11:14] = np.nan
temp_df.loc[temp_df['mean radius']==19.69, ['mean radius']]=np.nan
temp_df.head()

In [ ]:
# 결측치의 위치 표시하기
msno.matrix(temp_df)

In [ ]:
msno.__version__